In [ ]:
#必要なライブラリをインポート
import numpy as np
import pandas as pd
import re


class Timer:
    """
    Timer measures elapsed time taken to execute a code block.
    It is recommended to use this object with `with` block.
    
    Usage:
        with Timer(message):
            # code block to be measured here...
    """
    def __init__(self, msg: str):
        self.msg = msg
        
    def __enter__(self):
        print(self.msg, end='')
        self.start = time.time()
        return self

    def __exit__(self, *args):
        elapsed = time.time() - self.start
        print(' done: elapsed %.2f sec.' % elapsed)

In [ ]:
#東京都23区の物件を個別に保存しておいたものを読み込み、結合
csv_files = ['suumo_central_east.csv', 'suumo_west_north_south.csv']
data = [pd.read_csv(file, encoding='utf-8') for file in csv_files]
df = pd.concat(data, axis=0, ignore_index=True).reset_index(drop=True)

# id: 179873 の変な立地情報を削除する
df.loc[179873]['立地3'] = None

print('Shape of df:', df.shape)

#立地を「最寄駅」と「徒歩〜分」に分割
splitted1 = df['立地1'].str.split(' 歩', expand=True)
splitted1.columns = ['立地11', '立地12']
splitted2 = df['立地2'].str.split(' 歩', expand=True)
splitted2.columns = ['立地21', '立地22']
splitted3 = df['立地3'].str.split(' 歩', expand=True)
splitted3.columns = ['立地31', '立地32']

#その他費用を、「敷金」「礼金」「保証金」「敷引,償却」に分割
splitted4 = df['敷/礼/保証/敷引,償却'].str.split('/', expand=True)
splitted4.columns = ['敷金', '礼金', '保証金', '敷引償却']

#分割したカラムを結合
df = pd.concat([df, splitted1, splitted2, splitted3, splitted4], axis=1)

#分割前のカラムは分析に使用しないので削除しておく
df.drop(['立地1','立地2','立地3','敷/礼/保証/敷引,償却'], axis=1, inplace=True)

#数値として扱いたいので、不要な文字列を削除
df['築年数'] = df['築年数'].str.replace('新築', '0') #新築は築年数0年とする
df['築年数'] = df['築年数'].str.replace(r'築(\d+)年', r'\1')
# df['築年数'] = df['築年数'].str.replace('年', '')
df['賃料'] = df['賃料'].str.replace(u'万円', u'')
df['管理費'] = df['管理費'].str.replace(u'円', u'')
df['専有面積'] = df['専有面積'].str.replace('m2', '')
df['立地12'] = df['立地12'].str.replace(u'分', u'')
df['立地22'] = df['立地22'].str.replace(u'分', u'')
df['立地32'] = df['立地32'].str.replace(u'分', u'')
df['敷金'] = df['敷金'].str.replace(u'万円', u'')
df['礼金'] = df['礼金'].str.replace(u'万円', u'')
df['保証金'] = df['保証金'].str.replace(u'万円', u'')
df['敷引償却'] = df['敷引償却'].str.replace(u'万円', u'')
df['敷引償却'] = df['敷引償却'].replace('実費', 0) #「実費」と文字列が入っている場合がある

df.head()

In [ ]:
#「-」を0に変換
df['管理費'] = df['管理費'].replace('-', 0)
df['敷金'] = df['敷金'].replace('-', 0)
df['礼金'] = df['礼金'].replace('-', 0)
df['保証金'] = df['保証金'].replace('-', 0)
df['敷引償却'] = df['敷引償却'].replace('-', 0)

#文字列から数値に変換
df['賃料'] = pd.to_numeric(df['賃料'])
df['管理費'] = pd.to_numeric(df['管理費'])
df['敷金'] = pd.to_numeric(df['敷金'])
df['礼金'] = pd.to_numeric(df['礼金'])
df['保証金'] = pd.to_numeric(df['保証金'])
df['敷引償却'] = pd.to_numeric(df['敷引償却'])
df['築年数'] = pd.to_numeric(df['築年数'])
df['専有面積'] = pd.to_numeric(df['専有面積'])
df['立地12'] = pd.to_numeric(df['立地12'])
df['立地22'] = pd.to_numeric(df['立地22'])
df['立地32'] = pd.to_numeric(df['立地32'])

#単位を合わせるために、管理費以外を10000倍。
df['賃料'] = df['賃料'] * 10000
df['敷金'] = df['敷金'] * 10000
df['礼金'] = df['礼金'] * 10000
df['保証金'] = df['保証金'] * 10000
df['敷引償却'] = df['敷引償却'] * 10000

#管理費は実質的には賃料と同じく毎月支払うことになるため、「賃料+管理費」を家賃を見る指標とする
df['賃料+管理費'] = df['賃料'] + df['管理費']
#敷金/礼金と保証金は同じく初期費用であり、どちらかが適用されるため、合計を初期費用を見る指標とする
df['敷/礼/保証金'] = df['敷金'] + df['礼金'] + df['保証金']

df.head()

In [ ]:
#住所を「東京都」「〜区」「市町村番地」に分割
splitted6 = df['住所'].str.split('区', expand=True)
splitted6.columns = ['区', '市町村']
splitted6['区'] = splitted6['区'] + '区'
splitted6['区'] = splitted6['区'].str.replace('東京都', '')

#立地を「路線」「駅」「徒歩〜分」に分割
splitted7 = df['立地11'].str.split('/', expand=True)
splitted7.columns = ['路線1', '駅1']
splitted7['徒歩1'] = df['立地12']
splitted8 = df['立地21'].str.split('/', expand=True)
splitted8.columns = ['路線2', '駅2']
splitted8['徒歩2'] = df['立地22']
splitted9 = df['立地31'].str.split('/', expand=True)
splitted9.columns = ['路線3', '駅3']
splitted9['徒歩3'] = df['立地32']

#結合
df = pd.concat([df, splitted6, splitted7, splitted8, splitted9], axis=1)
#不要なカラムを削除
df.drop(['立地11','立地12','立地21','立地22','立地31','立地32'], axis=1, inplace=True)

#階を数値化。地下はマイナスとして扱う
splitted10 = df['階'].str.split('-', expand=True)
splitted10.columns = ['階1', '階2']
splitted10['階1'] = splitted10['階1'].str.replace(u'階', u'')
splitted10['階1'] = splitted10['階1'].str.replace(u'B', u'-')
splitted10['階1'] = pd.to_numeric(splitted10['階1'])

df = pd.concat([df, splitted10], axis=1)

#建物高さを数値化。地下は無視。
df['建物高さ'] = pd.to_numeric(df['建物高さ'].str.replace(r'(地下\d+地上)?(\d+)階建', r'\2').str.replace(u'平屋', u'1'))

#indexを振り直す（これをしないと、以下の処理でエラーが出る）
df = df.reset_index(drop=True)
df

In [ ]:
#間取りを「部屋数」「DK有無」「K有無」「L有無」「S有無」に分割
df['間取りDK'] = 0
df['間取りK'] = 0
df['間取りL'] = 0
df['間取りS'] = 0
df['間取り'] = df['間取り'].str.replace(u'ワンルーム', u'1') #ワンルームを1に変換

for symb in ['DK', 'K', 'L', 'S']:
    df.loc[df['間取り'].str.contains(symb), '間取り' + symb] = 1

df['間取り'] = df['間取り'].str.replace(r'[DKLS]', '')
df['間取り'] = pd.to_numeric(df['間取り'])
df

In [ ]:
#カラムを入れ替えて、csvファイルとして出力
df = df[['マンション名','住所','区','市町村','間取り','間取りDK','間取りK','間取りL','間取りS','築年数','建物高さ','階1',
         '専有面積','賃料+管理費','敷/礼/保証金','路線1','駅1','徒歩1','路線2','駅2','徒歩2','路線3','駅3','徒歩3',
         '賃料','管理費','敷金','礼金','保証金','敷引償却']]

df.to_csv('suumo_tokyo.csv', encoding='utf-8')